In [ ]:
import json
import requests
import re
from pprint import pprint
import pandas as pd

In [ ]:
env_path = ".env"

In [ ]:
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [ ]:
api_key = env['cc_api_key']
token = env['cc_token']

To get the token put this in the web browser: https://api.cc.email/v3/idfed?client_id={api_key}&redirect_uri=https://localhost&response_type=token&scope=contact_data

In [ ]:
f"https://api.cc.email/v3/idfed?client_id={api_key}&redirect_uri=https://localhost&response_type=token&scope=contact_data"

In [ ]:
def executeRequest(url, headers, payload):
    
    try:
    ## Create empty object first
        r_json = {}
    
    ## cleaner way of using requests
        response = requests.get(url, headers=headers, data=payload)

    except:
        print("There was an problem in the request :(")
        return None

    ## always nice to print the url as a sanity check
    print(response.url)

    # if succesful, populate your response json
    if  response.status_code == 200:
            r_json = response.json()
    else:
        print(f'Failed to get data {response.status_code}, {response.json()}')
    
    return r_json

In [ ]:
headers = {
  'Authorization': f'Bearer {token}'
}
payload = {}
url = "https://api.cc.email/v3/contact_lists?include_count=false"

In [ ]:
r_contact_lists = executeRequest(url, headers, payload)

In [ ]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
    except:
        print("There was a problem with the string.")
    return reg_exp

In [ ]:
def searchContactListsID(r_json, reg_exp):
    try:
        l_json = r_json.get("lists", [])
        assert type(l_json) is list and len(l_json) is not 0,  "Error with l_json"
        sel_contact_dict = {
        d['name']: d.get('list_id', '')
        for d in l_json
        if re.search(reg_exp, d['name']) != None
        }        
    except:
        print("There was a problem with the structure of the json")
    return sel_contact_dict

In [ ]:
contact_lists_of_interest = ["Educator Ambassadors"]

In [ ]:
testing_val = contact_lists_of_interest[0]

In [ ]:
cl_to_search = create_strict_reg_exp(testing_val)

In [ ]:
id_dict = searchContactListsID(r_json = r_contact_lists, to_search = cl_to_search)

In [ ]:
def requestURLbyListID(id_contact_list):
    try:
        url = f"https://api.cc.email/v3/contacts?lists={id_contact_list}&include=street_addresses&limit=500&include_count=false"
    except:
        print("There was a problem with the id.")
    return url

In [ ]:
url_contacts = requestURLbyListID(id_dict[testing_val])

In [ ]:
headers = {
  'Authorization': f'Bearer {token}'
}
payload = {}

In [ ]:
r_contacts = executeRequest(url_contacts, headers, payload)

In [ ]:
def getContactsLocation(r_json):
    try:
        contacts_list = []
        l_json = r_json.get("contacts", [])
        for d in l_json:
            street_address = d.get('street_addresses', [{}])#[0]
            if street_address:
                street_address = street_address[0]
                postal_code = street_address.get("postal_code", None)
                country = street_address.get("country", None)
                if postal_code and country:
                    contact_dict = {
                        'contact_id': d.get('contact_id', ''), ## Need a fallback for contact_id?
                        'postal_code': postal_code,
                        'country': country
                    }
                    contacts_list.append(contact_dict)  
        df = pd.DataFrame(contacts_list)
    except:
        print("There was a problem with the structure of the json")
    return df

In [ ]:
contacts_location_df = getContactsLocation(r_contacts)

In [ ]:
def missLocation(r_json, df):
    l_json = r_json.get("contacts", [])
    all_contacts_list = []
    for d in l_json:
        all_contacts_list.append(d.get('contact_id', ''))
    original_set = set(all_contacts_list)
    located_set = set(df.contact_id)
    contact_diff = original_set.difference(located_set)
    return contact_diff

In [ ]:
contacts_without_location = missLocation(r_contacts,contacts_location_df)

In [ ]:
len(contacts_without_location)

In [ ]:
def writeLocationCsv(df, csvName):
    csv_file = f'./{csvName}.csv'
    df.to_csv(csv_file, index=False)
    print(f"{csv_file} written")

In [ ]:
writeLocationCsv(contacts_location_df, testing_val)

# Once the csv is ready it can be published in arcgis online via the arcgis api

In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

In [ ]:
aol_password = env['aol_password']
aol_username = env['aol_username']

In [ ]:
#getting into the GIS
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [ ]:
csv_item = gis.content.add({}, csv_file)
csv_lyr = csv_item.publish(None,  { 'CountryCode' : 'country',
                                    'Postal' : 'postal_code'} )

In [ ]:
new_folder_details = gis.content.create_folder("constant_contact")
csv_item.move(new_folder_details)
csv_lyr.move(new_folder_details)
csv_lyr.share(everyone = True)
#print(csv_lyr.ownerFolder)